In [1]:
%%writefile ./home_network.py 
import random

#---------------------------------------------------------------------------
#Author: Dr.Harsha Padmanabhan Fan: padm0012 Date_Completed: 31 October 2025
#---------------------------------------------------------------------------

"""  This module implements an intelligent home automation system that controls smart devices
     based on environmental conditions, occupancy, and time schedules to optimise energy consumption"""

#-------------------------------------------------
#Stage1 Basic Frame work desgin and implementation
#-------------------------------------------------
""" Stage 1 Configure devices like smart light, smart plug, automated shutters, 
    and smart thermostat for responsive control"""

#----------------------
#Device Class - Parent
#----------------------
class Device: 
    """ Base class for all smart devices  """
    def __init__(self, name, power = 0.0): 
        self.name = name
        self.status = "off"
        self.power = power   #initialises a new device, with all devices starting in off state

    def turn_on(self):
        self.status = "on" #turn on device
        
    def turn_off(self):
        self.status = "off" # turn off device
        
    def toggle(self):
        if self.status == "on": #toggel state of device between on and off
            self.turn_off()
        else:
            self.turn_on()
            
    def reset(self):
        self.status ="off" # reset device
       
    def random_set(self):
        self.status = random.choice(["on","off"]) # set random status to device to either on or off
       
    def get_status(self): 
        return self.status # get status of device

    def get_power(self): 
        return self.power if self.status == "on" else 0.0 # calculate power of device if device is on

    def is_accessible(self):
        return self.status == "on"  # check if device is accessible

#---------------------------------------
#Specific devices (inheritance) - Child
#---------------------------------------
#All Specific devices inherit from parent class Device
class SmartLight(Device):
    """Smart light with adjustable brightness and colour mode"""
    def __init__(self, name, power = 10.0, brightness = 100, colour = "warm"):
        super().__init__(name, power)
        self.brightness = brightness
        self.colour = colour
        self.base_power = power

    def set_brightness(self,level):
        self.brightness = max(0, min(100, level)) #brightness betwee 0 to 100%)
        print("%s brightness set to %d%%." %(self.name, self.brightness))

    def set_colour(self,colour):
        if colour in ["auto", "warm white", "bright white"]: #colour adjustable to auto, warm and bright white
            self.colour = colour 
            print("%s colour set to %s." %(self.name, self.colour))
        else:
            print("Invalid colour mode for %s." %self.name)

    def get_power(self):
        """power use depends on brightness"""
        if self.status == "off":
            return 0.0

        brightness_factor = self.brightness/100.0 # power varies depending on colour
        if self.colour == "warm white":
            colour_factor = 1.0
        elif self.colour == "bright white":
            colour_factor = 1.2
        else: #auto mode is most efficient
            colour_factor = 0.9 

        total_power = self.base_power * brightness_factor * colour_factor
        return total_power 
              

class SmartThermostat(Device):
        """updates temperature and switches mode automatically"""
        def __init__(self,name, power = 1500.0, target_temp =22.0):
            super().__init__(name, power)
            self.target_temp = target_temp # desired temperature
            self.current_temp = 22.0 #start at room temperature
            self.mode = "off"
            self.base_power = power

        def update_temperature(self,current_temp):
            self.current_temp = current_temp
            if self.current_temp <self.target_temp - 0.5: #if temp below 0.5C start heating 
                self.mode = "heating"
                self.turn_on()
                
            elif self.current_temp > self.target_temp + 0.5: #if temp above 0.5C start cooling 
                self.mode = "cooling"
                self.turn_on()
                
            else: 
                self.mode = "off" # if within comfort range turn off thermostat
                self.turn_off()
               
        def get_power(self):
            if self.status == "off":
                return 0.0
            #heating uses 100% cooling uses 120%
            if self.mode == "heating":
                return self.power
            elif self.mode == "cooling":
                return self.power * 1.2
            return 0.0
            
        def get_status(self):
            if self.status == "off":
                return "off"
            if self.power < self.base_power and self.mode in ["heating", "cooling"]:
                return "on" #return on when runing at reduced power othewise return the mode
            if self.mode in ["heating", "cooling"]:
                return self.mode
            return "on"            
        

class SmartPlug(Device):
    """Smart plug can control attached devices"""
    def __init__(self, name, power = 0.2):
        super().__init__(name, power)
        self.attached_device = None # no device initially attached

    def attach(self, device): #attach a device for control through the plug
        self.attached_device = device

    def turn_on(self):
        super().turn_on() #turn on plug itself
        if self.attached_device:
            self.attached_device.turn_on() #turn on attached devices
        
    def turn_off(self): #turn off plug
        super().turn_off()
        if self.attached_device:
            self.attached_device.turn_off() #turn off attached devices
                
    def get_power(self): #return power of attached device if plug is on and deviceis attached 
        if self.attached_device and self.status == "on":
            return self.attached_device.get_power()
        return 0.0
            
class AutomatedWindowShutter(Device):
    """ Automated shutters that open/close and consume power only while moving """
    def __init__(self, name, power = 5.0, move_duration = 2.0):
        super().__init__(name, power)
        self.position = "closed"
        self.is_moving = False
        self.move_duration = move_duration #seconds it takes to move
        self.last_energy_used = 0.0 
        self.total_energy_used = 0.0

    def open_shutter(self): #opens shutter
        self.is_moving = True
        self.turn_on()
        print("%s is OPENING..." %self.name)
        
        self.last_energy_used =(self.power * self.move_duration)/ 3600.0  #simulating energy used for motion
        self.total_energy_used += self.last_energy_used 
        
        self.position = "open"
        self.is_moving = False
        self.turn_off()
        print("%s is now OPEN." %self.name)

    def close_shutter(self): #closes shutter
        self.is_moving = True
        self.turn_on()
        print("%s is CLOSING..." %self.name)

        self.last_energy_used =(self.power * self.move_duration)/ 3600.0 
        self.total_energy_used += self.last_energy_used 
        
        self.position = "closed"
        self.is_moving = False
        self.turn_off()
        print("%s is now CLOSED." %self.name)

    def get_power(self): #gets power if shutter moves
        return self.power if self.is_moving else 0.0

    def get_last_energy_used(self): #gets power of most recent opening 
        return self.last_energy_used

    def get_total_energy_used(self): #gets cumulative power used 
        return self.total_energy_used   
        
class Sensor(Device):
    """ Base class for sensors """
    def __init__(self, name, sensor_type, power = 0.5):
        super().__init__(name, power)
        self.sensor_type = sensor_type
        self.value = None
        self.min_range = None
        self.max_range = None
        self.unit ="N/A"
        self.status = "off"

    def calibrate(self, min_range, max_range, unit): # to set range and unit 
        self.min_range = min_range
        self.max_range = max_range
        self.unit = unit 
       
    def validate_reading (self): # to ensure valid data 
        if self.value is None:
            return False
        return self.min_range <= self.value <= self.max_range

    def get_power(self):
        if self.status.lower() != "on":
            return 0.0
        return self.power
           

#----------------------------------------------------------
#Motion and Temperature sensors (inheritance) - Grandchild 
#----------------------------------------------------------
class MotionSensor(Sensor):
    """ detects motion, sensitivity set from 0.0 to 0.1"""
    def __init__(self, name, sensitivity =0.5):
        super().__init__(name, sensor_type = "motion") 
        self.sensitivity = sensitivity 
        self.status = "off"

    def detect_motion(self):
        return random.random() <self.sensitivity #returns true if motion detected based on sensitivity
        

class TemperatureSensor(Sensor):
    """ detects temperature """
    def __init__(self, name, ambient_temp = 25.0):
        super().__init__(name, sensor_type = "temperature")
        self.value = ambient_temp # current temp in °C
        self.calibrate(-10, 45,"°C") #sets valid range -10°C to 45°C
        self.status = "off"

    def update_temperature(self, new_temp):
        self.value = new_temp #stores new temp
       

#----------------------------------
#HomeAutomationSystem (composition)
#----------------------------------
class HomeAutomationSystem:
    def __init__(self):
        self.light =SmartLight("Living Room Light", brightness =80)
        self.smart_thermostat = SmartThermostat("Smart Thermostat", target_temp = 22.0)
        self.plug = SmartPlug("Coffee Plug")
        self.shutter = AutomatedWindowShutter("Bedroom Shutter")
        self.motion = MotionSensor("Hallway Motion", sensitivity = 0.8)
        self.temp_sensor = TemperatureSensor("Outdoor Temp", ambient_temp = 28)

        self.devices = [self.light, self.smart_thermostat, self.plug,self.shutter, self.motion, self.temp_sensor]

        self.turn_all_off()
        
    def turn_all_on(self): #turns all devices on
        for d in self.devices:
            d.turn_on()
                
    def turn_all_off(self): #turns all devices off
        for d in self.devices:
            d.turn_off()
                
    def reset_all(self): #resets all devices to default off state
        for d in self.devices:
            d.reset()
        
    def randomise_all(self): #randomly sets devices to on or off
        for d in self.devices:
            d.random_set()

    def show_status(self): #shows the status of all devices
        for d in self.devices:
            print("%-25s   | %3s"%(d.name, d.get_status())) 

    def get_total_power_consumption(self):
        """ calculates and displays total power consumed across all devices"""
        total = 0.0
        for d in self.devices:
            total += d.get_power()
        return total     


Overwriting ./home_network.py


In [2]:
from home_network import *

#---------------------------------
#Home Automation System Simulation
#---------------------------------
""" Simulation testing demonstrates device behaviours collectively under random and reset conditions. """
def main ():
    print("Home Automation System Simulation")
    print("-" * 39)
    devices =[
        SmartLight('Bedroom Light'),
        SmartLight('Kitchen Light'),
        SmartThermostat('Smart Thermostat device'),
        SmartPlug('Coffee Plug'),
        AutomatedWindowShutter('Bedroom Shutter'),
        TemperatureSensor('Outdoor Sensor')
    ]

    print ('Running simulation of random settings:\n--------------------------------------')
    """ iterate through all devices and output each one's name, status and if it is currently accessible"""
    for d in devices:
        d.random_set()
        status = d.get_status()
        accessible = "Yes" if status =="on" else "No"
        print("%-25s   | Status: %-15s | %s" %(d.name, str(status), d.is_accessible()))

   
    print("-" * 39)    
    print ('\nRunning simulation of reset:\n--------------------------------------') 
    """iterate through all devices, RESET each and then output each one's name, status and if it is currently accessible"""
    for d in devices:
        d.reset()
        status = d.get_status()
        accessible = "Yes" if status == "on" else "NO"
        print("%-25s   | Status: %-15s | %s" %(d.name, str(status), d.is_accessible()))
    
if __name__=="__main__":
        main()


Home Automation System Simulation
---------------------------------------
Running simulation of random settings:
--------------------------------------
Bedroom Light               | Status: on              | True
Kitchen Light               | Status: off             | False
Smart Thermostat device     | Status: off             | False
Coffee Plug                 | Status: off             | False
Bedroom Shutter             | Status: on              | True
Outdoor Sensor              | Status: off             | False
---------------------------------------

Running simulation of reset:
--------------------------------------
Bedroom Light               | Status: off             | False
Kitchen Light               | Status: off             | False
Smart Thermostat device     | Status: off             | False
Coffee Plug                 | Status: off             | False
Bedroom Shutter             | Status: off             | False
Outdoor Sensor              | Status: off             | Fal

In [3]:
%%writefile ./home_network_tests.py
#-----------------------------------------------------
# UNIT TESTS for Stage 1 - Home Automation System
#-----------------------------------------------------
import unittest
from home_network import *
import random

class TestHomeAutomationSystem(unittest.TestCase):
    """Comprehensive test suite for Home Automation System. 
       This class aggregates all devices into a single cohesive system using composition """

    def setUp(self):
        
        self.light = SmartLight("Bedroom Light", brightness=75)
        self.smart_thermostat = SmartThermostat("Main Thermostat", target_temp=22.0)
        self.smart_plug = SmartPlug("Desk Plug")
        self.shutter = AutomatedWindowShutter("Living Room Shutter")
        self.motion_sensor = MotionSensor("Hallway Motion", sensitivity=0.9)
        self.temp_sensor = TemperatureSensor("Outdoor Temperature", ambient_temp=30)

        
        self.home = HomeAutomationSystem()

    # -------------------------
    # Reset Tests
    # -------------------------
    """ resets all devices"""
    def test_reset_smart_light(self):
        self.light.turn_on()
        self.light.reset()
        self.assertEqual(self.light.get_status(), "off")
    
    def test_reset_smart_thermostat(self):
        self.smart_thermostat.turn_on()
        self.smart_thermostat.reset()
        self.assertEqual(self.smart_thermostat.get_status(), "off")

    def test_reset_smart_plug(self):
        self.smart_plug.turn_on()
        self.smart_plug.reset()
        self.assertEqual(self.smart_plug.get_status(), "off")

    def test_reset_shutter(self):
        self.shutter.turn_on()
        self.shutter.reset()
        self.assertEqual(self.shutter.get_status(), "off")

    def test_reset_motion_sensor(self):
        self.motion_sensor.turn_on()
        self.motion_sensor.reset()
        self.assertEqual(self.motion_sensor.get_status(), "off")
    
    def test_reset_temp_sensor(self):
        self.temp_sensor.turn_on()
        self.temp_sensor.reset()
        self.assertEqual(self.temp_sensor.get_status(), "off")

    # -------------------------
    # Random setting tests
    # -------------------------
    def test_random_set_status(self):
        """Ensure random_set changes device status randomly """
        statuses = set()
        for i in range(10):
            self.light.random_set()
            statuses.add(self.light.get_status())
            #should have both on and off after several runs
        self.assertTrue("on" in statuses and "off" in statuses)

    def test_random_setting_for_all_devices(self):
        """ Ensure random setting across all devices """
        for device in [self.light, self.smart_thermostat, self.smart_plug,
                       self.shutter, self.motion_sensor, self.temp_sensor]:
            device.random_set()
            self.assertIn(device.get_status(),["on","off"])
            
    # -------------------------------------------------
    # Power & Status Tests
    # -------------------------------------------------
    def test_turn_all_on(self):
        """ Turn all devices on """
        self.home.turn_all_on()
        for d in self.home.devices:
            self.assertEqual(d.get_status(), "on")

    def test_turn_all_off(self):
        """ Turn all devices off """
        self.home.turn_all_off()
        for d in self.home.devices:
            self.assertEqual(d.get_status(), "off")

    def test_total_power_all_off(self): 
        """ turn all power off to test total power consumed"""
        self.home.turn_all_off()
        total = self.home.get_total_power_consumption()
        self.assertEqual(total,0.0)

    def test_total_power_all_on(self):
        """ turn all power on to test total power consumed"""
        self.home.turn_all_on()
        total = self.home.get_total_power_consumption()
        self.assertGreater(total,0.0)

    # -------------------------------------------------
    # Device-specific extreme condition
    # -------------------------------------------------
    def test_smartlight_extreme_brightness(self):
        """ Brightness should clamp between 0 and 100 """
        self.light.set_brightness(-50)
        self.assertEqual(self.light.brightness,0)
        self.light.set_brightness(150)
        self.assertEqual(self.light.brightness,100)

    def test_smart_thermostat_temperature_modes(self):
        """ Heating and cooling should switch modes correctly """
        self.smart_thermostat.update_temperature(18.0)
        self.assertEqual(self.smart_thermostat.mode,"heating")
        
        self.smart_thermostat.update_temperature(25.0)
        self.assertEqual(self.smart_thermostat.mode,"cooling")

        self.smart_thermostat.update_temperature(22.0)
        self.assertEqual(self.smart_thermostat.mode,"off")

    def test_shutter_open_close(self):
        """ Window shutter should open and close correctly """
        self.shutter.open_shutter()
        self.assertEqual(self.shutter.position,"open")
        self.shutter.close_shutter()
        self.assertEqual(self.shutter.position,"closed")

    def test_motion_sensor_dection(self):
        """ Motion sensor should return boolean detection """
        result = self.motion_sensor.detect_motion()
        self.assertIsInstance(result,bool)

    def test_temperature_sensor_update_and_validation(self):
        """ Temperature sensor should update and validate correctly """
        self.temp_sensor.update_temperature(26.5)
        self.assertAlmostEqual(self.temp_sensor.value, 26.5, delta = 0.1)
        valid = self.temp_sensor.validate_reading()
        self.assertTrue(valid)

    def test_smart_plug_power_and_attachment(self):
        """ Smart plug should report power of attached device only when on """
        self.smart_plug.attach(self.light)
        self.smart_plug.turn_on()
        self.light.turn_on()
        self.assertGreater(self.smart_plug.get_power(),0.0)
        self.smart_plug.turn_off()
        self.assertEqual(self.smart_plug.get_power(),0.0)

if __name__ =="__main__":
    unittest.main()
        
    

Overwriting ./home_network_tests.py


In [4]:
%run home_network_tests.py

..................
----------------------------------------------------------------------
Ran 18 tests in 0.017s

OK


Living Room Shutter is OPENING...
Living Room Shutter is now OPEN.
Living Room Shutter is CLOSING...
Living Room Shutter is now CLOSED.
Bedroom Light brightness set to 0%.
Bedroom Light brightness set to 100%.


In [5]:
%%writefile ./home_network.py 
from datetime import datetime
import random
#-------------------------------------------
# Stage 2 : System extention and innovation
#-------------------------------------------   
class EnergyEfficientHome(HomeAutomationSystem):
    """ The EnergyEfficientHome class inherits from the HomeAutomationSystem and 
        introduces automatic energy optimisation to minimise total power usage"""

    def __init__(self):
        super().__init__()
        
        #attaching coffee machine to SmartPlugs from parent class
        self.coffee_machine = Device("Coffee Machine", power =800.0)
        self.plug.attach(self.coffee_machine)

        motion = self.motion.detect_motion()
        outdoor_temp = self.temp_sensor.value
        
    #-------------------------------------------------
    # Energy Optimization Logic
    #-------------------------------------------------

    def optimise_energy_usage(self):
        """  This function implements automated control logic for all devices. 
        It reduces energy waste by ensuring device only operate when needed, 
        depending on ennironmental conidtions and time windows"""

        
        ### Optimise energy usage base on sensor data and conditions###
        motion = self.motion.detect_motion()
        outdoor_temp = self.temp_sensor.value
        current_hour = datetime.now().hour
        
        #SmartLight: Turn off when no motion, dims to 60% when motion is detected, comes on between 6pm to 6 am
        if motion == False:
            self.light.turn_off()
            
        else:
            if current_hour >= 18 or current_hour < 6:
                self.light.turn_on()
                self.light.set_brightness(60)
                print("Lights on, motion detected between 6PM to 6 AM")
                
            else:
                self.light.turn_off()
        

        #SmartThermostat : only ON if people are home and adjusted for partial operation to save power
          
        if motion == False:
            self.smart_thermostat.turn_off()

        else:
            temp_diff = abs(outdoor_temp - self.smart_thermostat.target_temp)
                
            self.smart_thermostat.update_temperature(outdoor_temp)
            current_mode = self.smart_thermostat.mode

            if current_mode != "off":
                base_power = self.smart_thermostat.power
                if temp_diff < 3:
                    reduced_power = base_power *0.5  # detects change of  less than 3°C from target temp , SmartThermostat  works at 50% base power 
                else:
                   reduced_power = base_power * 0.75 #if it detects change of more than 3°C from target temp SmartThermostat works at 75% base power
    
                self.smart_thermostat.power = reduced_power
                print("Thermostat ON (%s is on, temperature difference = %0.1f°C, reduced power : %.0f W)" %(current_mode,temp_diff, reduced_power))
            else:
                print ("Thermostat OFF (Temperature within comfort range)")
                
        #Smart Plug : Coffee machine control, turn off when no motion detected and between 6 am to 12 pm to save power        
                  
        if motion == False:
            self.plug.turn_off()
            self.coffee_machine.turn_off()
        else:
            if 6 <= current_hour < 12:
                self.plug.turn_on()
                self.coffee_machine.turn_on()
                print("Coffee machine active (%02d:00, within 6 -12 AM window)" %current_hour)
            
            else:
                self.plug.turn_off()
                self.coffee_machine.turn_off()
                print("Coffee machine inactive (%02d:00, outside 6 -12 AM window)" %current_hour)
                     
                  
        #Automated shutter: Closed at night for privacy and closes when (>30°C); opens when cool (<25°C)
        if motion and (current_hour >= 19 or current_hour < 6):
            self.shutter.close_shutter()
            print("Shutter is closed for privacy at night")
            
        elif outdoor_temp > 30:
            self.shutter.close_shutter()
    
        elif outdoor_temp < 25:
            self.shutter.open_shutter()

        else:
            self.shutter.turn_off()
            self.shutter.is_moving = False 
    
        print("Energy optimisation completed")
        
    #-------------------------------------------------
    # Energy Comparison Utility
    #-------------------------------------------------    
    #comparing total power before and after optimisation
    def optimise_and_compare(self):
        before = self.get_total_power_consumption()
        self.optimise_energy_usage()
        after = self.get_total_power_consumption()

        #re-setting shutter state after measurement
        self.shutter.is_moving = False
        self.shutter.turn_off()

        print("Before: %0.2f W   | After: %0.2f W"  %(before, after))
        return before, after
         


Overwriting ./home_network.py


In [6]:
import importlib
import home_network
importlib.reload(home_network)
from datetime import datetime
from home_network import *

#---------------------------------
#Energy Efficient Home Simulation
#---------------------------------
"""The second simulation extends the EnergyEfficientHome system to 
   test optimisation logic under realistic conditions"""

def main():
    print("\n=======================================")
    print("ENERGY OPTIMISATION SYSTEM SIMULATION")
    print("=======================================")

    current_time = datetime.now().strftime("%I:%M %p")    
    print("Current Time : %s" %current_time)

    
    system = EnergyEfficientHome()

    # power to motion and temperature sensors remains on at all times, both during periods of motion and inactivity, to simulate real world
    system.motion.turn_on()  
    system.temp_sensor.turn_on()
    
    
    print("\nSetting up realistic initial conditions...")
       
    system.light.turn_on()
    system.smart_thermostat.update_temperature(20.0)
    system.plug.turn_on()
        
    print("\nInitial Device Statuses:")
    print("-" * 39)
    for d in system.devices:
        print("%-30s | Status: %-10s | Power: %6.1f W" % (d.name, d.get_status(), d.get_power()))
    
    print("\nCalculating total initial power consumption...")
    total_before = system.get_total_power_consumption()
    print("Total Power (Before Optimisation): %.2f W" % total_before)
    
    # Simulating environment
    print("\n" + "-" * 45)
    print("Simulating environment conditions and sensors")
    print("-" * 45)
    
    outdoor_temp = random.uniform(15, 35)
    system.temp_sensor.update_temperature(outdoor_temp)
    print(" Outdoor temperature: %.2f °C" % system.temp_sensor.value)
    
    motion_state = system.motion.detect_motion()
    print(" Motion detected: %s" % ("Yes" if motion_state else "No"))

    original_detect = system.motion.detect_motion()
    system.motion.detect_motion = lambda : motion_state
    
    print("\nRunning energy optimisation...\n")
    
   
    before, after = system.optimise_and_compare()
    
    
    print("\n" + "-" * 45)
    print("Post-optimisation summary")
    print("-" * 45)
    for d in system.devices:
        print("%-30s | Status: %-10s | Power: %6.1f W" % (d.name, d.get_status(), d.get_power()))
    
    print("\nTotal Power Before Optimisation: %.2f W" % before)
    print("Total Power After Optimisation: %.2f W" % after)
    
    print("\nOptimisation result:", end=" ")
    if after < before:
        savings = before - after
        percentage = (savings / before) * 100
        print("Power reduced by %.2f W (%.1f%% savings)" % (savings, percentage))
    elif after == before:
        print("No change in total consumption")
    else:
        print("Power increased by %.2f W (check optimisation logic)." % (after - before))
    

if __name__ == "__main__":
    main()


ENERGY OPTIMISATION SYSTEM SIMULATION
Current Time : 04:19 PM

Setting up realistic initial conditions...

Initial Device Statuses:
---------------------------------------
Living Room Light              | Status: on         | Power:    7.2 W
Smart Thermostat               | Status: heating    | Power: 1500.0 W
Coffee Plug                    | Status: on         | Power:  800.0 W
Bedroom Shutter                | Status: off        | Power:    0.0 W
Hallway Motion                 | Status: on         | Power:    0.5 W
Outdoor Temp                   | Status: on         | Power:    0.5 W

Calculating total initial power consumption...
Total Power (Before Optimisation): 2308.20 W

---------------------------------------------
Simulating environment conditions and sensors
---------------------------------------------
 Outdoor temperature: 18.59 °C
 Motion detected: Yes

Running energy optimisation...

Thermostat ON (heating is on, temperature difference = 3.4°C, reduced power : 1125 W)
Cof

In [7]:
%%writefile  ./home_network_tests.py

from home_network import *
import unittest

#-----------------------
# Unit test for Stage 2 
#-----------------------
class TestEnergyEfficientHome(unittest.TestCase):
    """ Unit test 2, provides a structured verification of specific functionalities within energy optimisation system """
    def setUp(self):
        self.system = EnergyEfficientHome()
        self.system.motion.turn_on()
        self.system.temp_sensor.turn_on()
        
    def test_lights_turn_off_without_motion(self): 
        """ testing lights turn off without motion """
        self.system.light.turn_on()
        self.system.motion.sensitivity = 0.0 
        self.system.optimise_energy_usage()
        self.assertEqual(self.system.light.get_status(),"off")

    def test_light_dim_at_night_with_motion(self): 
        """testing lights dims at night with motion"""
        
        self.system.motion.sensitivity = 1.0 
        self.system.light.turn_on()
        self.system.light.set_brightness(100)
        current_hour = datetime.now().hour
        self.system.optimise_energy_usage()
        
        if current_hour >=18 or current_hour < 6:
            self.assertEqual(self.system.light.get_status(),"on")
            self.assertEqual(self.system.light.brightness, 60)
        else:
            self.assertEqual(self.system.light.get_status(),"off")

    def test_coffee_machine_active_morning(self): 
        """testing coffee machine active between 6 AM to 12 PM  with motion detected"""
        self.system.motion.sensitivity = 1.0
        self.system.plug.turn_off()
        current_hour = datetime.now().hour
        self.system.optimise_energy_usage()
        if  6 <= current_hour < 12:
            self.assertEqual(self.system.plug.get_status(),"on")
            self.assertEqual(self.system.coffee_machine.get_status(),"on")
        else:
            self.assertEqual(self.system.plug.get_status(),"off")
            self.assertEqual(self.system.coffee_machine.get_status(),"off")
    
    def test_coffee_machine_without_motion(self): 
        """testing coffee machine inactive without motion detected"""
        self.system.motion.sensitivity = 0.0
        self.system.plug.turn_on()
        self.system.coffee_machine.turn_on()
        self.system.optimise_energy_usage()
        self.assertEqual(self.system.plug.get_status(),"off")
        self.assertEqual(self.system.coffee_machine.get_status(),"off")
         
    def test_shutter_open_when_cool(self): 
        """testing shutter open when cool and day time""" 
        self.system.motion.sensitivity = 1.0
        self.system.temp_sensor.update_temperature(22)
        self.system.shutter.close_shutter()
        current_hour = datetime.now().hour
        self.system.optimise_energy_usage()
        if (current_hour >=19 or current_hour <6 ):
            self.assertEqual(self.system.shutter.position, "closed")
        else:
            self.assertEqual(self.system.shutter.position, "open")
            
    def test_shutter_closes_when_hot(self): 
        """ensures shutter closes automatically when hot"""
        self.system.motion.sensitivity = 1.0
        self.system.temp_sensor.update_temperature(35)
        self.system.shutter.open_shutter()
        self.system.optimise_energy_usage()
        self.assertEqual(self.system.shutter.position, "closed")

    def test_shutter_closes_at_night(self): 
        """ensure shutter closed between 7 PM and 6 AM for privacy"""
        self.system.motion.sensitivity = 1.0
        self.system.temp_sensor.update_temperature(22)
        self.system.shutter.open_shutter()
        current_hour = datetime.now().hour
        self.system.optimise_energy_usage()
        if (current_hour >=19 or current_hour <6 ):
            self.assertEqual(self.system.shutter.position, "closed")      
       
    def test_thermostat_turns_off_near_target(self): 
        """ensure thermostat turns off around target temperature"""
        self.system.motion.sensitivity = 1.0
        self.system.temp_sensor.update_temperature(22)
        self.system.smart_thermostat.turn_on()
        self.system.optimise_energy_usage()
        self.assertEqual(self.system.smart_thermostat.get_status(), "off")

    def test_thermostat_reduces_power(self): 
        """testing for snall differences in temp"""
        self.system.motion.sensitivity = 1.0
        self.system.temp_sensor.update_temperature(20)
        self.system.optimise_energy_usage()
        self.assertEqual(self.system.smart_thermostat.get_status(), "on")
        self.assertLess(self.system.smart_thermostat.power, self.system.smart_thermostat.base_power)

    def test_total_power_reduced_after_optimisation(self):
        """tests if total power comsumption is reduced after optimisation"""
        self.system.turn_all_on()
        self.system.smart_thermostat.turn_on()
        self.system.plug.turn_on()
        self.system.motion.sensitivity = 0.0
        self.system.temp_sensor.update_temperature(22)   
        before, after = self.system.optimise_and_compare() 
        self.assertLessEqual(after,before) 
        self.assertLess(after, 10.0)  
if __name__ =="__main__":
    unittest.main()


Overwriting ./home_network_tests.py


In [8]:
%run home_network_tests.py

..........
----------------------------------------------------------------------
Ran 10 tests in 0.010s

OK


Living Room Light brightness set to 60%.
Lights on, motion detected between 6PM to 6 AM
Thermostat ON (cooling is on, temperature difference = 6.0°C, reduced power : 1125 W)
Coffee machine inactive (23:00, outside 6 -12 AM window)
Bedroom Shutter is CLOSING...
Bedroom Shutter is now CLOSED.
Shutter is closed for privacy at night
Energy optimisation completed
Energy optimisation completed
Living Room Light brightness set to 100%.
Living Room Light brightness set to 60%.
Lights on, motion detected between 6PM to 6 AM
Thermostat ON (cooling is on, temperature difference = 6.0°C, reduced power : 1125 W)
Coffee machine inactive (23:00, outside 6 -12 AM window)
Bedroom Shutter is CLOSING...
Bedroom Shutter is now CLOSED.
Shutter is closed for privacy at night
Energy optimisation completed
Energy optimisation completed
Bedroom Shutter is OPENING...
Bedroom Shutter is now OPEN.
Living Room Light brightness set to 60%.
Lights on, motion detected between 6PM to 6 AM
Thermostat OFF (Temperature w